# A2: Vector Semantics

By Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik. Updated in 2025 by Ricardo Muñoz Sánchez

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for vectors work in approximating semantic similarity of phrases when compared to human judgements.

This lab uses code from a file called `dist_erk.py` which contains functions similar to those shown in the lecture. You can use either set of functions to solve these tasks.

In [ ]:
# The code for dist_erk.py uses both Spacy and NLTK, so make sure to have them installed!
# Our code also uses SciPY and scikit-learn, so you'll need to install it as well.
# If you're unsure how to do this, check out these websites:
### https://scipy.org/beginner-install/
### https://scikit-learn.org/stable/install.html
### https://spacy.io/usage
### https://www.nltk.org/install.html


# We also need to make sure we have the necessary models and datasets for Spacy
import spacy
spacy.cli.download('en_core_web_sm')
spacy.cli.download('en_core_web_lg')
# You only need to run this cell once
# You *need* to restart the kernel after downloading the model!

In [3]:
# the following command simply imports all the methods from the dist_erk file
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus `wikipedia.txt` stored in `wikipedia.zip`. This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/).

When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below. It may already exist in `/srv/data/computational-semantics/`.

In [4]:
# corpus_dir = './wikipedia/'
corpus_dir = '/srv/data/computational-semantics/assignments/wikipedia/'


## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls to functions from `dist_erk.py` which are similar to functions shown during the lecture.

Do not despair if the code takes a bit long to run!
It took me about 9 minutes for the cell below.

In [5]:
import numpy as np

numdims = 1000
svddim = 5

# Which words to use as targets and context words?
# We need to count the words and keep only the N most frequent ones
# Which function would you use here with which variable?
ktw = do_word_count(corpus_dir, numdims)

wi = make_word_index(ktw) # word index
words_in_order = sorted(wi.keys(), key=lambda w:wi[w]) # sorted words

# Create different spaces (the original matrix space, the ppmi space, the svd space)
# Which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir, wi, numdims)

print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k, wi)

print('svd transform')
svdspace_1k = svd_transform(ppmispace_1k, numdims, 5)
print('done.')

reading file wikipedia.txt
create count matrices
reading file wikipedia.txt


1145485it [01:30, 12678.93it/s]


ppmi transform
svd transform
done.


In [6]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])

house: [2551 3714 3104  567  962  627  443  185  311  189  131   28   93  169
   81  125  151  408  194   89   79   29  217  184   62   15   31   70
   10    1   41   21    1   31   37    1   30    5   25    7    3   20
   11    1   32   36    2    5   65    4    0   46    8   18   28    0
   20    7    8   16   10   40    0  175   10    2    7   19    1  174
   11    3    1    6    0    0    0   10    9   11    7   24    4    4
   14   23   58    7    0   10    2    3   10    6   18    6   13    3
   22    0    3    5    3    7   14    3   40   20   19   15    6    8
   23    4    5    1   19    0    3    1    0   14    0   14   53    7
    7   11    6    5    5    4   12    6   53    1    1  433    4    0
    5    7    7   12    1    1    3    4   17    8   16    1    2   31
    1   12   14    1   44    6   14    9   38    7    2    6    8    1
   10    6   10    1    9    7    9    4    3    9    0   11    3    2
    0    2   11   37    2    0    2    1    5    9   10   16    4    6

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. All matrices are available in the folder `pretrained` of the `wikipedia.zip`file. These are `ktw_wikipediaktw.npy`, `raw_wikipediaktw.npy`, `ppmi_wikipediaktw.npy`, `svd50_wikipedia10k.npy`. Make sure they are in your path as we load them below.

In [7]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load(f'{corpus_dir}/pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load(f'{corpus_dir}/pretrained/raw_wikipediaktw.npy', allow_pickle=True).tolist()
ppmispace_10k = np.load(f'{corpus_dir}/pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).tolist()
svdspace_10k  = np.load(f'{corpus_dir}/pretrained/svd50_wikipedia10k.npy', allow_pickle=True).tolist()
print('Done.')


Please wait...
Done.


In [8]:
# testing semantic space
print('house:', space_10k['house'])

house: [2554 3774 3105 ...    0    0    0]


## 3. Testing semantic similarity

The file `similarity_judgements.txt` contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The scores range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture [2]. You can find more details about how they were collected in the papers.

The following code will transform similarity scores into a Python-friendly format:

In [10]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # Checks if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw_10k))))
print('number of available word pairs to test:', len(word_pairs))
#list(zip(word_pairs, visual_similarity, semantic_similarity))

number of available words to test: 155
number of available word pairs to test: 774


We are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [the Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate the Spearman correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [11]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))

Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [12]:
raw_similarities  = [cosine(w1, w2, space_10k) for w1, w2 in word_pairs]
ppmi_similarities = [cosine(w1, w2, ppmispace_10k) for w1, w2 in word_pairs]
svd_similarities  = [cosine(w1, w2, svdspace_10k) for w1, w2 in word_pairs]

Calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [14]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, raw_similarities)
print(f"Semantic Similarity vs. Raw Similarity:\nrho = {rho:.4f}\np-value = {pval:.4f}\n\n")

rho, pval = stats.spearmanr(semantic_similarity, ppmi_similarities)
print(f"Semantic Similarity vs. PPMI Similarity:\nrho = {rho:.4f}\np-value = {pval:.4f}\n\n")

rho, pval = stats.spearmanr(semantic_similarity, svd_similarities)
print(f"Semantic Similarity vs. SVD Similarity:\nrho = {rho:.4f}\np-value = {pval:.4f}\n\n")


Semantic Similarity vs. Raw Similarity:
rho = 0.1522
p-value = 0.0000


Semantic Similarity vs. PPMI Similarity:
rho = 0.4547
p-value = 0.0000


Semantic Similarity vs. SVD Similarity:
rho = 0.4232
p-value = 0.0000




**Your answer should go here:**

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [15]:
from scipy import stats

rho, pval = stats.spearmanr(visual_similarity, raw_similarities)
print(f"Visual Similarity vs. Raw Similarity:\nrho = {rho:.4f}\np-value = {pval:.4f}\n\n")

rho, pval = stats.spearmanr(visual_similarity, ppmi_similarities)
print(f"Visual Similarity vs. PPMI Similarity:\nrho = {rho:.4f}\np-value = {pval:.4f}\n\n")

rho, pval = stats.spearmanr(visual_similarity, svd_similarities)
print(f"Visual Similarity vs. SVD Similarity:\nrho = {rho:.4f}\np-value = {pval:.4f}\n\n")

Visual Similarity vs. Raw Similarity:
rho = 0.1212
p-value = 0.0007


Visual Similarity vs. PPMI Similarity:
rho = 0.3838
p-value = 0.0000


Visual Similarity vs. SVD Similarity:
rho = 0.3097
p-value = 0.0000




**Your answer should go here:**

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions.

For example, we can perform `king - man` and add the resulting vector to `woman` and we hope to get the vector for `queen`. What would be the result of `stockholm - sweden + denmark`? Why? **[3 marks]**

If you want to learn more about vector differences between words (and words in analogy relations), check this paper [4].

**Your answer should go here:**

Here is some code that allows us to calculate such comparisons.

In [16]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [15]:
short = normalize(svdspace_10k['short'])
light = normalize(svdspace_10k['light'])
long = normalize(svdspace_10k['long'])
heavy = normalize(svdspace_10k['heavy'])

find_similar_to(light - (heavy - long), svdspace_10k)[:10]

[('long', np.float64(0.8733111261346902)),
 ('above', np.float64(0.8259671977311956)),
 ('around', np.float64(0.8030776291120686)),
 ('sun', np.float64(0.7692439111243974)),
 ('just', np.float64(0.767848197477811)),
 ('wide', np.float64(0.7672574319922534)),
 ('each', np.float64(0.7665960260861158)),
 ('circle', np.float64(0.7647746702909335)),
 ('length', np.float64(0.7601066921319761)),
 ('almost', np.float64(0.7542351860536627))]

**Your answer should go here:**

Find 5 similar pairs of pairs of words and test them. Hint: google for `word analogies examples`. You can also construct analogies that are not only lexical but also express other relations such as grammatical relations, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in `question-words.txt` from the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) described in [3]. Does the resulting vector similarity confirm your expectations? Remember you can only do this test if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [17]:
############## 1. Grammatical Category Relationship ##############
print(f"***************************************************")
see = normalize(svdspace_10k['see'])   
saw = normalize(svdspace_10k['saw'])
go = normalize(svdspace_10k['go'])
went = normalize(svdspace_10k['went'])

# Analogy computation: see - saw + went --> go
result_verb_tense = see - saw + went

# Find similar words to the result
similar_words = find_similar_to(result_verb_tense, svdspace_10k)[:10]
print(f"Target: go \n{similar_words}")
print(f"***************************************************\n\n")

''' 
############## 3. City-Country ##############
print(f"***************************************************")
city1 = normalize(svdspace_10k['Berlin'])
country1 = normalize(svdspace_10k['Germany'])
city2 = normalize(svdspace_10k['Beijing'])
country2 = normalize(svdspace_10k['China'])

# Analogy computation:  city1 - country1 + city2 --> country2
result_antonym = city1 - country1 + city2

# Find similar words to the result
similar_words = find_similar_to(result_antonym, svdspace_10k)
print(f"Target: {country2} \n{similar_words}")
print(f"***************************************************\n\n")
'''

'''
############## 4. Profession-to-Tool Relationship ##############
print(f"***************************************************")
doctor = normalize(svdspace_10k['doctor'])
stethoscope = normalize(svdspace_10k['stethoscope'])
carpenter = normalize(svdspace_10k['carpenter'])
hammer = normalize(svdspace_10k['hammer'])

# Analogy computation: doctor - stethoscope + hammer --> carpenter
result_profession_tool = doctor - stethoscope + hammer

# Find similar words to the result
similar_words = find_similar_to(result_profession_tool, svdspace_10k)
print(f"Target: carpenter \n{similar_words}")
print(f"***************************************************\n\n")
'''

############## 5. Spatial Relationship ##############
print(f"***************************************************")
above = normalize(svdspace_10k['above']) 
below = normalize(svdspace_10k['below'])
inside = normalize(svdspace_10k['inside'])
outside = normalize(svdspace_10k['outside'])

# Analogy computation: above - below + outside --> inside
result_spatial = above - below + outside

# Find similar words to the result
similar_words = find_similar_to(result_spatial, svdspace_10k)
print(f"Target: inside \n{similar_words}")
print(f"***************************************************\n\n")


############## 6. Comparative Relationship ##############
print(f"***************************************************")
big = normalize(svdspace_10k['big'])
bigger = normalize(svdspace_10k['bigger'])
light = normalize(svdspace_10k['light'])
lighter = normalize(svdspace_10k['lighter'])

# Analogy computation: big - bigger + lighter --> light
result_comparative = big - bigger + lighter

# Find similar words to the result
similar_words = find_similar_to(result_comparative, svdspace_10k)
print(f"Target: light \n{similar_words}")
print(f"***************************************************\n\n")

'''
############## 7. Cause and Effect Relationship ##############
print(f"***************************************************")
rain = normalize(svdspace_10k['rain'])
flood = normalize(svdspace_10k['flood'])
sun = normalize(svdspace_10k['sun'])
drought = normalize(svdspace_10k['drought'])

# Analogy computation: rain - flood + drought --> sun
result_cause_effect = rain - flood + drought

# Find similar words to the result
similar_words = find_similar_to(result_cause_effect, svdspace_10k)
print(f"Target: sun \n{similar_words}")
print(f"***************************************************\n\n")
'''

***************************************************
Target: go 
[('see', np.float64(0.8583508395881548)), ('disambiguation', np.float64(0.7768853234421607)), ('references', np.float64(0.7735965434839179)), ('article', np.float64(0.765212717517252)), ('external', np.float64(0.7548745794997013)), ('links', np.float64(0.7349505289046616)), ('link', np.float64(0.7337807566632727)), ('history', np.float64(0.7289093684415032)), ('page', np.float64(0.7189993467506255)), ('where', np.float64(0.7187067516796604))]
***************************************************


***************************************************
Target: inside 
[('outside', np.float64(0.9336832285217999)), ('where', np.float64(0.8279811720644288)), ('within', np.float64(0.8260974985007429)), ('main', np.float64(0.8127603811421519)), ('close', np.float64(0.7931313768308682)), ('through', np.float64(0.7926462281278546)), ('behind', np.float64(0.7852433291878936)), ('place', np.float64(0.785240806286174)), ('leaving', np.flo

'\n############## 7. Cause and Effect Relationship ##############\nprint(f"***************************************************")\nrain = normalize(svdspace_10k[\'rain\'])\nflood = normalize(svdspace_10k[\'flood\'])\nsun = normalize(svdspace_10k[\'sun\'])\ndrought = normalize(svdspace_10k[\'drought\'])\n\n# Analogy computation: rain - flood + drought --> sun\nresult_cause_effect = rain - flood + drought\n\n# Find similar words to the result\nsimilar_words = find_similar_to(result_cause_effect, svdspace_10k)\nprint(f"Target: sun \n{similar_words}")\nprint(f"***************************************************\n\n")\n'

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to examine how the composed vectors of phrases by different semantic composition functions/models introduced in [2] correlate with human judgements of similarity between phrases. We will use the dataset from this paper which is stored in `mitchell_lapata_acl08.txt`. If you are interested about further details about this task also refer to this paper.

(i) Process the dataset. The dataset contains human judgemements of similarity between phrases recorded one per line. The first column indicates the id of a participant making a judgement (`participant`), the next column is `verb`, followed by `noun` and `landmark`. From these three columns we can construct phrases that were compared by human informants, namely `verb noun` vs `verb landmark`. The next column `input` indicates a similarity score a participant assigned to a pair of such phrases on a scale from 1 to 7 where 1 is lowest and 7 is highest. The last column `hilo` groups the phrases into two sets: phrases where we expect low and phrases where we expect high similarity scores. This is because we want to test our compositional functions on two tasks and examine whether a function is discriminative between them. Correlation between scores could also be due to other reasons than semantic similarity and hence good prediction on both tasks simultaneously shows that a function is truly discriminating the phrases using some semantic criteria.

For extracting information you can use the code from the lecture to start with. How to structure this data is up to you - a dictionary-like format would be a good choice. Remember that each example was judged by several participants and phrases will repeat in the dataset. Therefore, you have to collect all judgments for a particular set of phrases and average them. This will become useful in step (iii).

(ii) Compose the vectors of the extracted word pairs by testing different compositional functions. In the lecture we introduced simple additive, simple multiplicative and combined models (details are described in [2]). Your task is to take a pair of phrases, e.g. the first example in the dataset `stray thought` and `stray roam` and for each phrase compute a composition of the vectors of their words using these functions, using one function per experiment run. For each phrase you will get a single vector. You can encode the words with any vector space introduced earlier (standard space, ppmi or svd) but your code should be structured in a way that it will be easy to switch between them. Finally, take the resulting (composed) vectors of phrase pairs in the dataset and calculate a cosine similarity between them.

(iii) Now you have cosine similairity scores between vectors of phrases but how do they compare with the average human scores that you calculated from the individual judgements from the `input` column of the dataset for the same phrases? Calculate Spearman rank correlation coefficient between two lists of the scores both for the `high` and the `low` task . 

We use the Spearmank rank correlation coefficient (or Spearman's rho) rather than Peason's correlation coefficent because we cannot compare cosine scores with human judgements directly. Cosine is a constinuous measure and human judgements are expressed as ranks. Also, we cannot say if 0.28 to 1 is the same (or different) to 6 to 7 in the human scores.  The Spearman rank correlation coeffcient turns the scores for all examples within each group first to ranks and then these ranks are correlated (or approximated to a linear function). 

In the end you should get a table similar to the one below from the paper. What is the best compositional function from those that you evaluated with your vector spaces and why?

<img src="res.png" alt="drawing" width="500"/>

Note that you might not get results in the same range as those in the paper.
That is ok, a good interpretation of results and discussion why sometimes they are not as good as you would expect is better than giving the best performing results with little to no analysis.


In [20]:
# (i) - Process the data
import pandas as pd
# Load the dataset
data_file = './mitchell_lapata_acl08.txt'
# df = pd.read_csv(data_file, sep=' ', header=None, names=["participant", "verb", "noun", "landmark", "input", "hilo"])
df = pd.read_csv(data_file, sep=' ', header=None, names=["participant", "verb", "noun", "landmark", "input", "hilo"], skiprows=1)

# Check the first few rows
print(df.head()[:10])

print(f"\nNumber of rows in the dataset: {len(df)}")


     participant   verb        noun landmark  input  hilo
0  participant20  stray     thought     roam      7   low
1  participant20  stray  discussion  digress      6  high
2  participant20  stray         eye     roam      7  high
3  participant20  stray       child  digress      1   low
4  participant20  throb        body    pulse      5  high

Number of rows in the dataset: 3600


In [27]:
# (ii) - Compose the vectors of the extracted word pairs by testing different compositional functions
# Did the paper point out any specific words vector models to use??
# TODO: Change to additive_composition(vector1, vector2) and multiplicative_composition(vector1, vector2) functions to avoid calculate word vectors multiple times in (iii)
import numpy as np
import spacy
import pandas as pd

# Load the spaCy model with pre-trained word embeddings
# nlp = spacy.load('en_core_web_lg') # Use this for larger models--run on Server
nlp = spacy.load('en_core_web_sm') # Use this for smaller models--run on laptop

# get word vector from model
# Here, we can easily change to other models
def get_word_vector(word):
    
    doc = nlp(word)
    word_vector = doc.vector
    
    
    # loaded the pretrained wikipidia model, miss a lot of words
    ''' 
    word_vector = space_10k[word] 
    # word_vector = ppmispace_10k[word]
    # word_vector = svdspace_10k[word]
    '''
    
    return word_vector

# compute additive composition of word vectors
def additive_composition(vector1, vector2):
    return vector1 + vector2

# compute multiplicative composition of word vectors
def multiplicative_composition(vector1, vector2):
    return vector1 * vector2  # Element-wise multiplication

# Compute the combined model by combining both additive and multiplicative compositions
def combined_composition(vector1, vector2, alpha=0.5, beta=0.5):
    # Compute additive and multiplicative compositions
    additive_vector = vector1 + vector2
    multiplicative_vector = vector1 * vector2
    # Combine using the weighted sum
    combined_vector = alpha * additive_vector + beta * multiplicative_vector

    return combined_vector


# Example usage for the first pair in the dataset
verb = df.iloc[0]['verb']
noun = df.iloc[0]['noun']
landmark = df.iloc[0]['landmark']

verb_vector = get_word_vector(verb)
noun_vector = get_word_vector(noun)
landmark_vector = get_word_vector(landmark)

additive_vector = additive_composition(verb_vector, noun_vector)
additive_landmark_vector = additive_composition(verb_vector, landmark_vector)

multiplicative_vector = multiplicative_composition(verb_vector, noun_vector)
multiplicative_landmark_vector = multiplicative_composition(verb_vector, landmark_vector)

combine_vector = combined_composition(verb_vector, noun_vector)
combine_landmark_vector = combined_composition(verb_vector, landmark_vector)

# Show results
print(f"Additive Composition Vector for {verb} + {noun} with {len(additive_vector)}: ", additive_vector)
print(f"Additive Composition Vector for {verb} + {landmark} with {len(additive_landmark_vector)}: ", additive_landmark_vector)

print(f"====================================")
print(f"Multiplicative Composition Vector for {verb} * {noun} with {len(multiplicative_vector)}: ", multiplicative_vector)
print(f"Multiplicative Composition Vector for {verb} * {landmark} with {len(multiplicative_landmark_vector)}: ", multiplicative_landmark_vector)


print(f"====================================")
print(f"Combined Composition Vector for {verb} + {noun} with {len(combine_vector)}: ", combine_vector)
print(f"Combined Composition Vector for {verb} + {landmark} with {len(combine_landmark_vector)}: ", combine_landmark_vector)



Additive Composition Vector for stray + thought with 96:  [-2.8077412e-01 -1.0771009e+00 -4.9315956e-01 -4.9916738e-01
 -1.7289631e+00 -9.1278744e-01 -6.4010036e-01  1.3711520e+00
 -1.3619955e+00 -5.1537853e-01  2.8763189e+00  4.2584985e-03
 -1.7156224e+00 -3.4795243e-01 -5.1936615e-01  7.9099011e-01
 -1.2130051e+00 -1.9159157e+00  1.4643033e+00  2.8166473e-03
 -2.4810183e-01  3.6704292e+00 -3.4811535e-01  6.2907076e-01
  5.3091413e-01  5.1060486e-01  2.4254048e+00  1.7724496e+00
 -1.8715836e+00 -1.1110920e+00  3.2693306e-01  1.5891888e+00
 -2.3967171e-01  9.1693723e-01 -2.2365990e+00 -1.3766204e+00
  1.3421464e-01  1.0027702e+00  2.3692086e-01  1.2133467e-01
  1.5892686e+00  6.2727970e-01  3.0477840e-01 -4.2670360e-01
  1.5327913e-01 -8.5817969e-01  2.6160359e+00 -1.9106920e+00
  3.8290650e-02  2.0223409e-02 -1.0205518e-01  1.9666293e-01
 -1.3328508e+00 -2.3903739e+00  1.3234867e+00  7.2310865e-01
  2.6132703e+00  4.7943795e-01  4.2628682e-01  9.3852776e-01
 -6.8670082e-01  1.2674949e

In [28]:
# (iii) - Compare the cosine similarity scores between vectors of phrases with the average human scores

from scipy.spatial.distance import cosine
from scipy.stats import spearmanr

# cosine similarity
def cosine_similarity(vec1, vec2):
    similarity = 1 - cosine(vec1, vec2)
    # print(f"Cosine Similarity: {similarity}")
    if similarity == np.nan:
        similarity = 0.0
    return  similarity

# compare the cosine similarity for each phrase pair with human input
human_scores_high = df[df['hilo'] == 'high']['input'].values
human_scores_low = df[df['hilo'] == 'low']['input'].values

# Collect the cosine similarities for the "high" and "low" tasks
hscore_high, hscore_low = [], []
add_cosin_sim_high, add_cosin_sim_low = [], []
multipli_cosin_sim_high, multipli_cosin_sim_low = [], []
combine_cosin_sim_high, combine_cosin_sim_low = [], []

for i, row in df.iterrows():
    verb = row['verb']
    noun = row['noun']
    landmark = row['landmark']
    verb_vector = get_word_vector(verb)
    noun_vector = get_word_vector(noun)
    landmark_vector = get_word_vector(landmark)
    
    ## additive composition
    add_com_vector = additive_composition(verb_vector, noun_vector)
    add_landmark_vector = additive_composition(verb_vector, landmark_vector)
    add_cosine_sim = cosine_similarity(add_com_vector, add_landmark_vector)
    
    ## multiplicative composition
    multipli_com_vector = multiplicative_composition(verb_vector, noun_vector)
    multipli_landmark_vector = multiplicative_composition(verb_vector, landmark_vector)
    multipli_cosine_sim = cosine_similarity(multipli_com_vector, multipli_landmark_vector)
    
    ## combined composition
    combine_com_vector = combined_composition(verb_vector, noun_vector, alpha=0.4, beta=0.6) # alpha=0.5, beta=0.5
    combine_landmark_vector = combined_composition(verb_vector, landmark_vector, alpha=0.4, beta=0.6)
    combine_cosine_sim = cosine_similarity(combine_com_vector, combine_landmark_vector)
    
    if row['hilo'] == 'high':
        hscore_high.extend([row['input']])
        add_cosin_sim_high.extend([add_cosine_sim])
        multipli_cosin_sim_high.extend([multipli_cosine_sim])
        combine_cosin_sim_high.extend([combine_cosine_sim])
    else:
        hscore_low.extend([row['input']])
        add_cosin_sim_low.extend([add_cosine_sim])
        multipli_cosin_sim_low.extend([multipli_cosine_sim])
        combine_cosin_sim_low.extend([combine_cosine_sim])

# Spearman rank correlation between cosine similarity and human scores
add_spearman_high = spearmanr(hscore_high, add_cosin_sim_high)
add_spearman_low = spearmanr(hscore_low, add_cosin_sim_low)
multipli_spearman_high = spearmanr(hscore_high, multipli_cosin_sim_high)
multipli_spearman_low = spearmanr(hscore_low, multipli_cosin_sim_low)
combine_spearman_high = spearmanr(hscore_high, combine_cosin_sim_high)
combine_spearman_low = spearmanr(hscore_low, combine_cosin_sim_low)

# Print Spearman correlations for both high and low tasks
data_print = {
    "Method": ["Additive", "Multiplicative", "Combined"],
    "High Correlation": [add_spearman_high.correlation, multipli_spearman_high.correlation, combine_spearman_high.correlation],
    "High p-value": [add_spearman_high.pvalue, multipli_spearman_high.pvalue, combine_spearman_high.pvalue],
    "Low Correlation": [add_spearman_low.correlation, multipli_spearman_low.correlation, combine_spearman_low.correlation],
    "Low p-value": [add_spearman_low.pvalue, multipli_spearman_low.pvalue, combine_spearman_low.pvalue]
}

# Create a DataFrame
df_print = pd.DataFrame(data_print)
print(df_print)


           Method  High Correlation  High p-value  Low Correlation  \
0        Additive          0.113951  1.124623e-06        -0.057074   
1  Multiplicative          0.189363  4.025974e-16         0.034722   
2        Combined          0.180484  9.214008e-15         0.022889   

   Low p-value  
0     0.015912  
1     0.142649  
2     0.333944  


**Any comments/thoughts should go here:**

# Literature

[1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

[2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
[3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

[4] E. Vylomova, L. Rimell, T. Cohn, and T. Baldwin. Take and took, gaggle and goose, book and read: Evaluating the utility of vector differences for lexical relation learning. arXiv, arXiv:1509.01692 [cs.CL], 2015.

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

## Marks

The assignment is marked on a 7-level scale where 4 is sufficient to complete the assignment; 5 is good solid work; 6 is excellent work, covers most of the assignment; and 7: creative work. 

This assignment has a total of 60 marks. These translate to grades as follows: 1 = 17% 2 = 34%, 3 = 50%, 4 = 67%, 5 = 75%, 6 = 84%, 7 = 92% where %s are interpreted as lower bounds to achieve that grade.